In [57]:
import numpy as np
import pandas as pd

# Load data
df = pd.read_excel("last_data.xlsx")

# Drop empty Excel artifact columns
df = df.loc[:, ~df.columns.str.contains("^Unnamed")]

# Sanity check
print(df.shape)
print(df.columns.tolist())


(78, 13)
['PBUG1', 'PBUG2', 'PBUG3', 'PBUG4', 'OUAI2', 'OUAI3', 'OUAI4', 'EST1', 'EST3', 'EST4', 'VUTAI1', 'VUTAI2', 'VUTAI3']


In [58]:
constructs = {
    "EST": ["EST1", "EST3", "EST4"],
    "OUAI": ["OUAI2", "OUAI3", "OUAI4"],
    "PBUG": ["PBUG1", "PBUG2", "PBUG3", "PBUG4"],
    "VUTAI": ["VUTAI1", "VUTAI2", "VUTAI3"]
}

all_indicators = [ind for inds in constructs.values() for ind in inds]


In [59]:
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [60]:
models = {
    "LinearRegression": Pipeline([
        ("scaler", StandardScaler()),
        ("model", LinearRegression())
    ]),

    "RidgeRegression": Pipeline([
        ("scaler", StandardScaler()),
        ("model", Ridge(alpha=1.0))
    ]),

    "RandomForest": RandomForestRegressor(
        n_estimators=300,
        random_state=42
    ),

    "GradientBoosting": GradientBoostingRegressor(
        n_estimators=300,
        random_state=42
    )
}


In [61]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)



In [62]:
results = []

valid_columns = set(df.columns)

for construct, indicators in constructs.items():

    # Inputs = indicators of all OTHER constructs
    input_features = [
        ind for ind in all_indicators
        if ind not in indicators and ind in valid_columns
    ]

    if len(input_features) == 0:
        raise ValueError(f"No input features found for construct {construct}")

    X = df[input_features]

    for target in indicators:

        if target not in valid_columns:
            continue

        y = df[target]

        for model_name, model in models.items():

            y_pred = cross_val_predict(model, X, y, cv=kf)

            # --- METRICS (version-safe) ---
            rmse = np.sqrt(mean_squared_error(y, y_pred))
            mae = mean_absolute_error(y, y_pred)
            r2 = r2_score(y, y_pred)

            # Q²predict (out-of-sample, mean baseline)
            q2 = 1 - np.sum((y - y_pred) ** 2) / np.sum((y - y.mean()) ** 2)

            results.append({
                "Construct": construct,
                "Indicator": target,
                "Model": model_name,
                "RMSE": rmse,
                "MAE": mae,
                "R2": r2,
                "Q2": q2
            })


In [63]:
# Convert results list to DataFrame
results_df = pd.DataFrame(results)

# Sanity check
print("Number of rows:", results_df.shape[0])
print(results_df.head())

# Save to CSV
results_df.to_csv(
    "Results/PLSpredict_Aligned_ML_Validation.csv",
    index=False
)

print("CSV file saved successfully.")


Number of rows: 52
  Construct Indicator             Model      RMSE       MAE        R2  \
0       EST      EST1  LinearRegression  0.725766  0.482785  0.041668   
1       EST      EST1   RidgeRegression  0.720152  0.479336  0.056435   
2       EST      EST1      RandomForest  0.695583  0.441133  0.119721   
3       EST      EST1  GradientBoosting  0.780053  0.492162 -0.107060   
4       EST      EST3  LinearRegression  0.663200  0.452701  0.332348   

         Q2  
0  0.041668  
1  0.056435  
2  0.119721  
3 -0.107060  
4  0.332348  
CSV file saved successfully.


In [64]:
import pandas as pd

ml_df = pd.read_csv("Results/PLSpredict_Aligned_ML_Validation.csv")

print(ml_df.shape)   # should be (56, 7)
ml_df.head()


(52, 7)


,Construct,Indicator,Model,RMSE,MAE,R2,Q2
0,EST,EST1,LinearRegression,0.725766,0.482785,0.041668,0.041668
1,EST,EST1,RidgeRegression,0.720152,0.479336,0.056435,0.056435
2,EST,EST1,RandomForest,0.695583,0.441133,0.119721,0.119721
3,EST,EST1,GradientBoosting,0.780053,0.492162,-0.107060,-0.107060
4,EST,EST3,LinearRegression,0.663200,0.452701,0.332348,0.332348


In [65]:
#select best model for each item
best_ml = (
    ml_df
    .sort_values("RMSE")
    .groupby(["Construct", "Indicator"], as_index=False)
    .first()
)

best_ml


,Construct,Indicator,Model,RMSE,MAE,R2,Q2
0,EST,EST1,RandomForest,0.695583,0.441133,0.119721,0.119721
1,EST,EST3,RandomForest,0.563280,0.394402,0.518374,0.518374
2,EST,EST4,RidgeRegression,0.631654,0.426659,0.262849,0.262849
3,OUAI,OUAI2,RandomForest,0.707710,0.489861,0.185244,0.185244
4,OUAI,OUAI3,RidgeRegression,0.865926,0.712839,0.148890,0.148890
5,OUAI,OUAI4,RandomForest,0.857866,0.701085,0.161532,0.161532
6,PBUG,PBUG1,RidgeRegression,0.638404,0.458961,0.355112,0.355112
7,PBUG,PBUG2,RandomForest,0.627323,0.415556,0.401585,0.401585
8,PBUG,PBUG3,RandomForest,0.821107,0.646272,0.135525,0.135525
9,PBUG,PBUG4,RidgeRegression,0.602035,0.491464,0.408498,0.408498


In [66]:
import os
from joblib import dump

# Create folder if it does not exist
MODEL_DIR = "Models"
os.makedirs(MODEL_DIR, exist_ok=True)

saved_models = {}

for _, row in best_ml.iterrows():

    construct = row["Construct"]
    indicator = row["Indicator"]
    model_name = row["Model"]

    # Skip missing ML cases (e.g., PBUG1, PBUG2)
    if pd.isna(model_name):
        print(f"Skipping {indicator} (no valid ML model)")
        continue

    # Define input features (same logic as training)
    input_features = [
        ind for ind in all_indicators
        if ind not in constructs[construct] and ind in df.columns
    ]

    X = df[input_features]
    y = df[indicator]

    # Get model template
    model = models[model_name]

    # Refit on FULL data
    model.fit(X, y)

    # File path (inside Models/)
    filename = os.path.join(
        MODEL_DIR,
        f"ML_BestModel_{construct}_{indicator}_{model_name}.joblib"
    )

    # Save model
    dump(model, filename)

    # Store metadata for later use
    saved_models[indicator] = {
        "construct": construct,
        "model_name": model_name,
        "features": input_features,
        "file": filename
    }

    print(f"Saved: {filename}")


Saved: Models/ML_BestModel_EST_EST1_RandomForest.joblib
Saved: Models/ML_BestModel_EST_EST3_RandomForest.joblib
Saved: Models/ML_BestModel_EST_EST4_RidgeRegression.joblib
Saved: Models/ML_BestModel_OUAI_OUAI2_RandomForest.joblib
Saved: Models/ML_BestModel_OUAI_OUAI3_RidgeRegression.joblib
Saved: Models/ML_BestModel_OUAI_OUAI4_RandomForest.joblib
Saved: Models/ML_BestModel_PBUG_PBUG1_RidgeRegression.joblib
Saved: Models/ML_BestModel_PBUG_PBUG2_RandomForest.joblib
Saved: Models/ML_BestModel_PBUG_PBUG3_RandomForest.joblib
Saved: Models/ML_BestModel_PBUG_PBUG4_RidgeRegression.joblib
Saved: Models/ML_BestModel_VUTAI_VUTAI1_RidgeRegression.joblib
Saved: Models/ML_BestModel_VUTAI_VUTAI2_RandomForest.joblib
Saved: Models/ML_BestModel_VUTAI_VUTAI3_RandomForest.joblib


In [67]:
#Code (PLSpredict table)
plspredict = pd.DataFrame({
    "Construct": [
        "EST","EST","EST",
        "OUAI","OUAI","OUAI",
        "PBUG","PBUG","PBUG","PBUG",
        "VUTAI","VUTAI","VUTAI"
    ],
    "Indicator": [
        "EST1","EST3","EST4",
        "OUAI2","OUAI3","OUAI4",
        "PBUG1","PBUG2","PBUG3","PBUG4",
        "VUTAI1","VUTAI2","VUTAI3"
    ],
    "PLS_Q2": [
        0.050,0.171,0.092,
        0.189,0.314,0.071,
        0.235,0.220,0.077,0.160,
        0.075,0.070,0.103
    ],
    "PLS_RMSE": [
        0.732,0.748,0.710,
        0.715,0.790,0.914,
        0.706,0.725,0.860,0.729,
        0.753,0.692,0.821
    ],
    "PLS_MAE": [
        0.551,0.561,0.505,
        0.567,0.634,0.723,
        0.569,0.520,0.736,0.600,
        0.623,0.546,0.662
    ]
})


In [68]:
#merge with ml
comparison = plspredict.merge(
    best_ml,
    on=["Construct", "Indicator"],
    how="left"
)

comparison = comparison.rename(columns={
    "Model": "Best_ML_Model",
    "RMSE": "ML_RMSE",
    "MAE": "ML_MAE",
    "Q2": "ML_Q2"
})

comparison


,Construct,Indicator,PLS_Q2,PLS_RMSE,PLS_MAE,Best_ML_Model,ML_RMSE,ML_MAE,R2,ML_Q2
0,EST,EST1,0.050,0.732,0.551,RandomForest,0.695583,0.441133,0.119721,0.119721
1,EST,EST3,0.171,0.748,0.561,RandomForest,0.563280,0.394402,0.518374,0.518374
2,EST,EST4,0.092,0.710,0.505,RidgeRegression,0.631654,0.426659,0.262849,0.262849
3,OUAI,OUAI2,0.189,0.715,0.567,RandomForest,0.707710,0.489861,0.185244,0.185244
4,OUAI,OUAI3,0.314,0.790,0.634,RidgeRegression,0.865926,0.712839,0.148890,0.148890
5,OUAI,OUAI4,0.071,0.914,0.723,RandomForest,0.857866,0.701085,0.161532,0.161532
6,PBUG,PBUG1,0.235,0.706,0.569,RidgeRegression,0.638404,0.458961,0.355112,0.355112
7,PBUG,PBUG2,0.220,0.725,0.520,RandomForest,0.627323,0.415556,0.401585,0.401585
8,PBUG,PBUG3,0.077,0.860,0.736,RandomForest,0.821107,0.646272,0.135525,0.135525
9,PBUG,PBUG4,0.160,0.729,0.600,RidgeRegression,0.602035,0.491464,0.408498,0.408498


In [69]:
comparison["RMSE_Delta"] = comparison["PLS_RMSE"] - comparison["ML_RMSE"]
comparison["Q2_Delta"] = comparison["ML_Q2"] - comparison["PLS_Q2"]

comparison


,Construct,Indicator,PLS_Q2,PLS_RMSE,PLS_MAE,Best_ML_Model,ML_RMSE,ML_MAE,R2,ML_Q2,RMSE_Delta,Q2_Delta
0,EST,EST1,0.050,0.732,0.551,RandomForest,0.695583,0.441133,0.119721,0.119721,0.036417,0.069721
1,EST,EST3,0.171,0.748,0.561,RandomForest,0.563280,0.394402,0.518374,0.518374,0.184720,0.347374
2,EST,EST4,0.092,0.710,0.505,RidgeRegression,0.631654,0.426659,0.262849,0.262849,0.078346,0.170849
3,OUAI,OUAI2,0.189,0.715,0.567,RandomForest,0.707710,0.489861,0.185244,0.185244,0.007290,-0.003756
4,OUAI,OUAI3,0.314,0.790,0.634,RidgeRegression,0.865926,0.712839,0.148890,0.148890,-0.075926,-0.165110
5,OUAI,OUAI4,0.071,0.914,0.723,RandomForest,0.857866,0.701085,0.161532,0.161532,0.056134,0.090532
6,PBUG,PBUG1,0.235,0.706,0.569,RidgeRegression,0.638404,0.458961,0.355112,0.355112,0.067596,0.120112
7,PBUG,PBUG2,0.220,0.725,0.520,RandomForest,0.627323,0.415556,0.401585,0.401585,0.097677,0.181585
8,PBUG,PBUG3,0.077,0.860,0.736,RandomForest,0.821107,0.646272,0.135525,0.135525,0.038893,0.058525
9,PBUG,PBUG4,0.160,0.729,0.600,RidgeRegression,0.602035,0.491464,0.408498,0.408498,0.126965,0.248498


In [70]:
comparison.to_csv(
    "Results/PLSpredict_vs_ML_BestModel_Comparison.csv",
    index=False
)


In [71]:
def compute_ai_governance_vision(
    df,
    items=("VUTAI1", "VUTAI2", "VUTAI3"),
    method="mean",
    new_col="AI_Governance_Vision",
    pct_col="AI_Governance_Vision_Pct",
    scale_min=1,
    scale_max=4
):
    """
    Compute the AI Governance Vision (AIGV) score and its percentage version.

    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame containing the VUTAI items.
    items : tuple or list
        Column names corresponding to VUTAI indicators.
    method : str
        Aggregation method: 'mean' or 'sum'.
    new_col : str
        Name of the column for the raw vision score.
    pct_col : str
        Name of the column for the percentage vision score.
    scale_min : int or float
        Minimum value of the Likert scale.
    scale_max : int or float
        Maximum value of the Likert scale.

    Returns
    -------
    pandas.DataFrame
        DataFrame with both vision score columns added.
    """

    # Safety checks
    missing = [col for col in items if col not in df.columns]
    if missing:
        raise ValueError(f"Missing VUTAI items in DataFrame: {missing}")

    if method == "mean":
        score = df[list(items)].mean(axis=1)
    elif method == "sum":
        score = df[list(items)].sum(axis=1)
    else:
        raise ValueError("method must be either 'mean' or 'sum'")

    # Raw score
    df[new_col] = score

    # Percentage transformation (scale-aware)
    df[pct_col] = ((score - scale_min) / (scale_max - scale_min)) * 100

    return df


In [72]:
df = compute_ai_governance_vision(df)

df[[
    "VUTAI1", "VUTAI2", "VUTAI3",
    "AI_Governance_Vision",
    "AI_Governance_Vision_Pct"
]]


,VUTAI1,VUTAI2,VUTAI3,AI_Governance_Vision,AI_Governance_Vision_Pct
0,4,4,4,4.000000,100.000000
1,4,4,4,4.000000,100.000000
2,4,4,4,4.000000,100.000000
3,4,4,4,4.000000,100.000000
4,1,1,1,1.000000,0.000000
...,...,...,...,...,...
73,4,4,3,3.666667,88.888889
74,3,3,2,2.666667,55.555556
75,3,4,3,3.333333,77.777778
76,4,4,4,4.000000,100.000000
